In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [4]:
dataset=pd.get_dummies(dataset, drop_first=True, dtype=int)

# dataset

In [6]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [7]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [8]:
depe=dataset[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(indep, depe, test_size=1/3, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'kernel':['linear', 'rbf', 'poly', 'sigmoid'], 'C':[10,100,1000,2000,3000], 'gamma':['auto', 'scale']}
classifier=GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=3, cv=5, n_jobs=-1, scoring='f1_weighted')
grid=classifier.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\Bagirathan\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [19]:
print(grid.best_params_)

{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}


In [12]:
grid_prediction=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_prediction)

In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_prediction)

In [23]:
from sklearn.metrics import f1_score
f1_mac=f1_score(y_test, grid_prediction,average='weighted')
print("The f1_mac value for best parameter {}:".format(grid.best_params_),f1_mac)

The f1_mac value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}: 0.9924946382275899


In [24]:
print("The confusion matrix:\n",cm)

The confusion matrix:
 [[51  0]
 [ 1 81]]


In [25]:
print("The CLF_report:\n",clf_report)

The CLF_report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [38]:
re=grid.cv_results_

In [39]:
table=pd.DataFrame.from_dict(re)


In [40]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013753,0.003213,0.014489,0.002534,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
1,0.019923,0.006518,0.016570,0.003098,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,1.000000,1.000000,0.988859,0.014751,4
2,0.020510,0.002454,0.014722,0.001639,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",1.000000,0.981014,1.000000,1.000000,0.981217,0.992446,0.009252,3
3,0.011321,0.002971,0.013939,0.001476,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.981569,1.000000,1.000000,0.981031,1.000000,0.992520,0.009163,1
4,0.014059,0.005309,0.014844,0.005961,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
5,0.015363,0.009416,0.017767,0.004351,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,1.000000,1.000000,0.988859,0.014751,4
6,0.021254,0.001909,0.010988,0.002355,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",1.000000,0.981014,1.000000,0.981031,0.981217,0.988652,0.009266,6
7,0.007071,0.002277,0.014953,0.003831,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.981569,1.000000,1.000000,0.981031,1.000000,0.992520,0.009163,1
8,0.011284,0.002313,0.015010,0.004181,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
9,0.016573,0.005817,0.013899,0.001989,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,0.981031,1.000000,0.985066,0.013807,9


In [41]:
age_input=float(input("Age:"))
bp_input=float(input("BP:"))
al_input=float(input("AL:"))
su_input=float(input("SU:"))
bgr_input=float(input("BGR:"))
bu_input=float(input("BU:"))
sc_input=float(input("SC:"))
sod_input=float(input("SOD:"))
pot_input=float(input("POT:"))
hrmo_input=float(input("HRMO:"))
pcv_input=float(input("PCV:"))
wc_input=float(input("WC:"))
rc_input=float(input("RC:"))
sg_b_input=float(input("SG_B:"))
sg_c_input=float(input("SG_C:"))
sg_d_input=float(input("SG_D:"))
sg_e_input=float(input("SG_E:"))
rbc_normal_input=float(input("RBC:"))
pc_normal_input=float(input("PC:"))
pcc_present_input=float(input("PCC:"))
ba_present_input=float(input("BA:"))
htn_yes_input=float(input("HTN:"))
dm_yes_input=float(input("DM:"))
cad_yes_input=float(input("CAD:"))
appet_yes_input=float(input("APPET:"))
pe_yes_input=float(input("PE:"))
ane_yes_input=float(input("ANE:"))

Age: 34
BP: 87.88
AL: 1
SU: 3
BGR: 219.33
BU: 57.98
SC: 4.33
SOD: 145.32
POT: 2.23
HRMO: 11.34
PCV: 35
WC: 7367
RC: 4.56
SG_B: 4
SG_C: 0
SG_D: 0
SG_E: 0
RBC: 1
PC: 1
PCC: 1
BA: 1
HTN: 0
DM: 1
CAD: 1
APPET: 1
PE: 1
ANE: 1


In [44]:
CKD_prediction=grid.predict([[age_input, bp_input, al_input, su_input, bgr_input, bu_input, sc_input, sod_input, pot_input, hrmo_input, pcv_input,
       wc_input, rc_input, sg_b_input, sg_c_input, sg_d_input, sg_e_input, rbc_normal_input, pc_normal_input,
       pcc_present_input, ba_present_input, htn_yes_input, dm_yes_input, cad_yes_input,
       appet_yes_input, pe_yes_input, ane_yes_input]])
print("CKD_Prediction{}:".format(CKD_prediction))

CKD_Prediction[0]:
